In [1]:
import escher
import cobra
import cobrakbase
%run ../../../scripts/bios_utils.py
%run ../../../scripts/escher_curation.py

cobrakbase 0.1.7


In [2]:
modelseed_local = cobrakbase.modelseed.from_local('/Users/fliu/workspace/jupyter/ModelSEEDDatabase')

load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/reactions.tsv
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/compounds.tsv
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Structures/Unique_ModelSEED_Structures.txt
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Compound_Aliases.txt
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_Aliases.txt
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_ECs.txt


In [4]:
ws = 'filipeliu:narrative_1504192868437'
kbase = cobrakbase.KBaseAPI('L3NXWHHX25VLEL6OAWB3Z5URDSDKZIMP')

In [226]:
kbase.save_object(id='Lipst1_1_NO_GP_DraftFilter.gf.Carbon-D-Glucose.fix', ws=ws, data=kmodel, otype='KBaseFBA.FBAModel')

[[283,
  'Lipst1_1_NO_GP_DraftFilter.gf.Carbon-D-Glucose.fix',
  'KBaseFBA.FBAModel-11.0',
  '2019-06-17T07:14:22+0000',
  1,
  'filipeliu',
  40036,
  'filipeliu:narrative_1548978605026',
  '8213bef0144e0d15dab2af333a4ff25e',
  1743918,
  {'Number gapgens': '0',
   'Type': 'MultipleGenomes',
   'Number gapfills': '1',
   'Source ID': 'fungi_merge_model_empty',
   'Number biomasses': '1',
   'Number compartments': '9',
   'Genome': '39957/29/5',
   'Source': 'sbml',
   'Number compounds': '2486',
   'Number reactions': '632',
   'Name': 'Lipomyces starkeyi'}]]

In [240]:
ws = 'filipeliu:narrative_1548978605026'
kmodel = kbase.get_object('Lipst1_1_NO_GP_DraftFilter.gf.Carbon-D-Glucose.fix', ws)
kmedia = kbase.get_object('Carbon-D-Glucose', ws)
#kgenome = 
kfba = kbase.get_object('Lipst1_1_NO_GP_DraftFilter.gf.fba', ws)
fbamodel = cobrakbase.core.model.KBaseFBAModel(kmodel)

In [5]:
#ecoli
kmodel = kbase.get_object('GCF_000005845.2.mdl', ws)
kmedia = kbase.get_object('Carbon-D-Glucose', ws)
#kgenome = 
kfba = kbase.get_object('GCF_000005845.2.fba', ws)

In [174]:
%run ../../../scripts/stoich_integration.py

In [217]:
def get_reaction_compartment(cmps):
    if not cmps == None:
        if len(cmps) == 1:
            return cmps.pop()
        elif 'e0' in cmps and len(cmps) == 2:
            return 'e0'
        elif 'r0' in cmps and len(cmps) == 2:
            return 'r0'
        elif 'c0' in cmps and len(cmps) == 2:
            cmps.remove('c0')
            return cmps.pop()
        elif len(cmps) == 0:
            print(reaction)
        print(cmps)
    else:
        return None

def get_compartments(r):
    compartments = set()
    for o in r.data['modelReactionReagents']:
        cpd_id = o['modelcompound_ref'].split('/')[-1]
        cpd = fbamodel.get_metabolite(cpd_id)
        cpd_cmp = cpd.data['modelcompartment_ref'].split('/')[-1]
        compartments.add(cpd_cmp)
    return compartments

def get_cstoichiometry(r):
    cstoichiometry = {}
    for o in r.data['modelReactionReagents']:
        cpd_id = o['modelcompound_ref'].split('/')[-1]
        cpd = fbamodel.get_metabolite(cpd_id)
        cpd_cmp = cpd.data['modelcompartment_ref'].split('/')[-1]
        coeff = o['coefficient']
        cstoichiometry[(cpd.id, cpd_cmp)] = coeff
    return cstoichiometry
        
get_cs(r)

{('cpd00009_c0', 'c0'): 1,
 ('cpd00087_c0', 'c0'): -1,
 ('cpd00002_c0', 'c0'): -1,
 ('cpd00201_c0', 'c0'): 1,
 ('cpd00047_c0', 'c0'): -1,
 ('cpd00008_c0', 'c0'): 1}

In [228]:
for i in range(len(kmodel['modelreactions'])):
    r = kmodel['modelreactions'][i]
    r['id'] = r['id'] + '_' + str(i)

In [229]:
def cluster_reactions2(rxn_to_hash, all_hashes):

    g = nx.Graph()
    for h in all_hashes:
        for h_val in all_hashes[h]:
            #print(h, h_val, all_hashes[h][h_val])
            for i in all_hashes[h][h_val]:
                g.add_edge(str(h_val) + '@HASH', i)
                    #print(prev, i)
            #break

    ccs = nx.algorithms.connected_components(g)
    ccs = [cc for cc in ccs]
    ccs = filter_hash(ccs)
    print('clusters:', len(ccs))
    
    return ccs

ccs = cluster_reactions2(rxn_to_hash, all_hashes)



clusters: 7


In [236]:
for cc in ccs:
    print()
    for id in cc:
        r = fbamodel.get_reaction(id)
        r.data['direction'] = '='
        r.data['maxforflux'] = 1000
        r.data['maxrevflux'] = 1000
        print(r.id, r.stoichiometry, r.get_bounds())


rxn00832_c_369 {'cpd00001_c0': -1, 'cpd00114_c0': -1, 'cpd02884_c0': 1} (1000, 1000, '=')
rxn00832_c_0 {'cpd00001_c0': -1, 'cpd00114_c0': -1, 'cpd02884_c0': 1} (1000, 1000, '=')

rxn00611_c_390 {'cpd00003_c0': -1, 'cpd00080_c0': -1, 'cpd00004_c0': 1, 'cpd00067_c0': 1, 'cpd00095_c0': 1} (1000, 1000, '=')
rxn00611_c_3 {'cpd00003_c0': -1, 'cpd00080_c0': -1, 'cpd00004_c0': 1, 'cpd00067_c0': 1, 'cpd00095_c0': 1} (1000, 1000, '=')

rxn03975_c_488 {'cpd00006_c0': -1, 'cpd01270_c0': -1, 'cpd00005_c0': 1, 'cpd00050_c0': 1, 'cpd00067_c0': 2} (1000, 1000, '=')
rxn03975_c_121 {'cpd00006_c0': -1, 'cpd01270_c0': -1, 'cpd00005_c0': 1, 'cpd00050_c0': 1, 'cpd00067_c0': 2} (1000, 1000, '=')

rxn00086_c_276 {'cpd00006_c0': -1, 'cpd00042_c0': -2, 'cpd00005_c0': 1, 'cpd00067_c0': 1, 'cpd00111_c0': 1} (1000, 1000, '=')
rxn00086_c_191 {'cpd00006_c0': -1, 'cpd00042_c0': -2, 'cpd00005_c0': 1, 'cpd00067_c0': 1, 'cpd00111_c0': 1} (1000, 1000, '=')

rxn00086_m_192 {'cpd00006_m0': -1, 'cpd00042_m0': -2, 'cpd00005

{'aliases': [],
 'dblinks': {},
 'direction': '<',
 'edits': {},
 'gapfill_data': {'gf.0': {'0': ['<', 1, []]}},
 'id': 'rxn00225_c0_537',
 'maxforflux': 1000000,
 'maxrevflux': 1000000,
 'modelReactionProteins': [],
 'modelReactionReagents': [{'coefficient': -1,
   'modelcompound_ref': '~/modelcompounds/id/cpd00002_c0'},
  {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00196_c0'},
  {'coefficient': -1, 'modelcompound_ref': '~/modelcompounds/id/cpd00029_c0'},
  {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00008_c0'}],
 'modelcompartment_ref': '~/modelcompartments/id/c0',
 'name': 'ATP:acetate phosphotransferase_c0',
 'numerical_attributes': {},
 'probability': 0,
 'protons': 0,
 'reaction_ref': '~/template/reactions/id/rxn00225_c',
 'string_attributes': {}}

In [224]:
for i in range(len(kmodel['modelreactions'])):
    r = kmodel['modelreactions'][i]
    r['id'] = rename[r['id']]

In [221]:
delete = set()
for cc in ccs:
    cc.pop()
    delete |= cc
fbamodel.delete_reactions(delete, store=True)
rename = {}
for rxn in fbamodel.reactions:
    seed_id = rxn.data['reaction_ref'].split('/')[-1].split('_')[0]
    cmps = get_compartments(rxn)
    cmp = None
    cmp = get_reaction_compartment(cmps)
    rename[rxn.id] = "{}_{}".format(seed_id, cmp)
    

(632, 632)

In [ ]:
for r in fbamodel.reactions:

In [182]:
h_func = hasher
rxn_to_hash = {}
all_hashes = {}
for r in fbamodel.reactions:
    cstoichiometry = get_cs(r)
    hashes = h_func(cstoichiometry)
    rxn_to_hash[r.id] = hashes
    for h in hashes:
        hash_val = hashes[h]
        if not h in all_hashes:
            all_hashes[h] = {}
        if not hash_val in all_hashes[h]:
            all_hashes[h][hash_val] = set()
        all_hashes[h][hash_val].add(r.id)

In [142]:
fbamodel.get_reaction('rxn00086_c')

In [151]:
fbamodel = cobrakbase.core.model.KBaseFBAModel(kmodel)
ids = set()
print(len(fbamodel.reactions))
for r in fbamodel.reactions:
    ids.add(r.id)
    if 'rxn00086_c' in r.id:
        print(r.id)

639


In [241]:
media_const = cobrakbase.convert_media(kmedia)
cobra_model = cobrakbase.convert_kmodel(kmodel, media_const)
model_json = cobra.io.to_json(cobra_model)
fbamodel = cobrakbase.core.model.KBaseFBAModel(kmodel)
cobra_model

Name,kbase
Memory address,0x0133fcfa978
Number of metabolites,935
Number of reactions,833
Number of groups,0
Objective expression,1.0*bio1_biomass - 1.0*bio1_biomass_reverse_6e711
Compartments,"c, m, x, e, v, r"


In [242]:
fba = cobrakbase.core.model.KBaseFBASolution(kfba)
fba

In [243]:
flux_data = {'rxn01643_c0' : -100}
for rxn_var in fba.data['FBAReactionVariables']:
    flux = rxn_var['value']
    rxn_id = rxn_var['modelreaction_ref'].split('/')[-1]
    flux_data[rxn_id] = flux
for rxn_var in fba.data['FBABiomassVariables']:
    flux = rxn_var['value']
    rxn_id = rxn_var['biomass_ref'].split('/')[-1]
    flux_data[rxn_id + '_biomass'] = flux
for rxn_var in fba.data['FBACompoundVariables']:
    flux = rxn_var['value']
    cpd_id = rxn_var['modelcompound_ref'].split('/')[-1]
    flux_data['EX_' + cpd_id] = -1 * flux

In [244]:
map_list = list(filter(lambda d : d['organism'] == 'BIOS', escher.list_cached_maps()))

In [245]:
map_list

[{'organism': 'BIOS', 'map_name': 'bios7.proteins'},
 {'organism': 'BIOS', 'map_name': 'bios7.Quinones'},
 {'organism': 'BIOS', 'map_name': 'bios7.fa'},
 {'organism': 'BIOS', 'map_name': 'bios7.AA'},
 {'organism': 'BIOS', 'map_name': 'bios7.LipidIV'},
 {'organism': 'BIOS', 'map_name': 'bios7.thf'},
 {'organism': 'BIOS', 'map_name': 'bios7.Central'},
 {'organism': 'BIOS', 'map_name': 'bios7.mycolate'},
 {'organism': 'BIOS', 'map_name': 'bios7.Riboflavin'},
 {'organism': 'BIOS', 'map_name': 'bios7.sugars'},
 {'organism': 'BIOS', 'map_name': 'bios7.f430'},
 {'organism': 'BIOS', 'map_name': 'bios7.Thiamine'},
 {'organism': 'BIOS', 'map_name': 'bios7.Nucleotides'},
 {'organism': 'BIOS', 'map_name': 'bios7.peptidoglycan'}]

In [10]:
ESCHER_HOME = '/Users/fliu/workspace/jupyter/data/escher'
map_id = 'bios7.Central'
escher_model_data = read_json(ESCHER_HOME + '/models/' + 'BIOS' + '/' + 'bios7.json')
escher_map_data = read_json(ESCHER_HOME + '/maps/' + 'BIOS' + '/' + map_id + '.json')
escher_map = EscherMap(escher_map_data)
escher_model = EscherModel(escher_model_data)

In [255]:
for reaction in cobra_model.reactions:
    print(reaction.compartments)
    break
reaction

{'c'}


Reaction identifier,rxn01740_c0
Name,Shikimate:NADP+ 3-oxidoreductase
Memory address,0x013409b7e10
Stoichiometry,cpd00006_c0 + cpd00383_c0 <=> cpd00005_c0 + cpd00067_c0 + cpd01716_c0 NADP + Shikimate <=> NADPH + H+ + 3-Dehydroshikimate
GPR,(gene_1556)
Lower bound,-1000
Upper bound,1000


In [252]:
for modelreaction in fbamodel.reactions:
    modelreaction.
    break

In [246]:
def fetch_metabolites_and_reactions(fbamodel, compartment_match = 'c0', compartment_match2 = 'c'):
    rxn_map_to_model = {}
    compartment_match2 = 'c'

    for modelreaction in fbamodel.reactions:
        reaction_ref = modelreaction.data['reaction_ref'].split('/')[-1]
        seed_id, compartment = reaction_ref.split('_')
        if compartment == compartment_match2:
            rxn_map_to_model[seed_id] = modelreaction.id
        #print(modelreaction.id, seed_id, compartment)

    compartment_match = 'c0'
    cpd_map_to_model = {}
    for metabolite in fbamodel.metabolites:
        seed_id = metabolite.data['compound_ref'].split('/')[-1]
        compartment = metabolite.data['modelcompartment_ref'].split('/')[-1]
        if compartment == compartment_match:
            cpd_map_to_model[seed_id] = metabolite.id
            
    return cpd_map_to_model, rxn_map_to_model

def build_escher_map(escher_map, model_json):
    map_json_str = json.dumps(escher_map.escher_map)
    builder = escher.Builder(map_json=map_json_str, model_json=model_json, reaction_data=flux_data)
    builder.set_highlight_missing(True)
    builder.set_enable_tooltips(True)
    builder.set_show_gene_reaction_rules(True)
    builder.set_and_method_in_gene_reaction_rule(True)
    return builder

In [247]:
cpd_map_to_model, rxn_map_to_model = fetch_metabolites_and_reactions(fbamodel)
print(len(rxn_map_to_model), len(cpd_map_to_model))

513 1267


In [249]:
cpd_remap, rxn_remap = escher_model.map_escher_model_data(cpd_map_to_model, rxn_map_to_model)

error {'rxn06108', 'rxn20240'} {'rxn38178', 'rxn20240', 'rxn06108', 'rxn20247', 'rxn33889', 'rxn38176', 'rxn28008', 'rxn20254', 'rxn28005'}
error {'cpd11420', 'cpd27735'} {'cpd11420', 'cpd27735'}
error {'cpd28060', 'cpd11421'} {'cpd28060', 'cpd11421'}
error {'cpd15283', 'cpd27794'} {'cpd15283', 'cpd27794'}
error {'cpd11476', 'cpd15277'} {'cpd11476', 'cpd15277'}
error {'cpd12458', 'cpd15268', 'cpd16828'} {'cpd12458', 'cpd15268', 'cpd16828'}
error {'cpd19013', 'cpd00013'} {'cpd19013', 'cpd00013'}
error {'cpd11466', 'cpd27551'} {'cpd27551', 'cpd11466'}
error {'cpd03521', 'cpd29676'} {'cpd03521', 'cpd29676'}
error {'cpd16855', 'cpd11482'} {'cpd16855', 'cpd11482'}
error {'cpd28300', 'cpd11665'} {'cpd28300', 'cpd11665'}
error {'cpd28301', 'cpd11669'} {'cpd28301', 'cpd11669'}


In [192]:
escher_map.swap_ids(cpd_remap, rxn_remap)

In [250]:
for o in map_list:
    map_id = o['map_name']
    escher_map_data = read_json(ESCHER_HOME + '/maps/' + 'BIOS' + '/' + map_id + '.json')
    escher_map = EscherMap(escher_map_data)
    escher_map.swap_ids(cpd_remap, rxn_remap)
    builder = build_escher_map(escher_map, model_json)
    builder.save_html('../bin/escher/maps/' + map_id, overwrite=True)

In [194]:
cobra_model.summary()

IN FLUXES             OUT FLUXES          OBJECTIVES
--------------------  ------------------  ------------------
cpd00013_e0  100      cpd00179_e0  66     bio1_biomass  20.7
cpd00027_e0    5      cpd11416_c0  20.7
cpd00009_e0    4.33   cpd11791_e0   4.41
cpd00048_e0    2.52
cpd00067_e0    2.41
cpd10515_e0    0.305
cpd00030_e0    0.102
cpd00058_e0    0.102
cpd00099_e0    0.102
cpd00205_e0    0.102
cpd00254_e0    0.102
cpd00034_e0    0.102
cpd00063_e0    0.102
cpd00149_e0    0.102
cpd10516_e0    0.102


In [66]:
#builder.display_in_notebook(enable_editing=True)

In [48]:
mm = json.loads(model_json)

In [68]:
for r in mm['reactions']:
    if 'bio' in r['id']:
        print(r['id'])

bio1_biomass


In [73]:
rxn_var

{'biomass_ref': '~/fbamodel/biomasses/id/bio1',
 'class': 'unknown',
 'lowerBound': 0,
 'max': 1000,
 'min': 0,
 'other_max': [],
 'other_min': [],
 'other_values': [],
 'upperBound': 1000,
 'value': 0.697833,
 'variableType': 'biomassflux'}

In [72]:
fba.data.keys()



dict_keys(['FBABiomassVariables', 'FBACompoundBounds', 'FBACompoundVariables', 'FBAConstraints', 'FBADeletionResults', 'FBAMetaboliteProductionResults', 'FBAMinimalMediaResults', 'FBAMinimalReactionsResults', 'FBAPromResults', 'FBAReactionBounds', 'FBAReactionVariables', 'FBATintleResults', 'MFALog', 'PROMKappa', 'QuantitativeOptimizationSolutions', '__VERSION__', 'additionalCpd_refs', 'allReversible', 'biomassRemovals', 'biomassflux_objterms', 'calculateReactionKnockoutSensitivity', 'comboDeletions', 'compoundflux_objterms', 'decomposeReversibleDrainFlux', 'decomposeReversibleFlux', 'defaultMaxDrainFlux', 'defaultMaxFlux', 'defaultMinDrainFlux', 'drainfluxUseVariables', 'fbamodel_ref', 'findMinimalMedia', 'fluxMinimization', 'fluxUseVariables', 'fva', 'gapfillingSolutions', 'geneKO_refs', 'id', 'inputfiles', 'maximizeActiveReactions', 'maximizeObjective', 'media_list_refs', 'media_ref', 'minimizeErrorThermodynamicConstraints', 'minimize_reaction_costs', 'minimize_reactions', 'noErrorT

In [164]:
#modelseed_local.get_seed_compound('cpd15268').data['name']

In [171]:

#map_reaction

In [19]:
ls /Users/fliu/workspace/jupyter/data/escher/maps/BIOS

bios4.Porphyrin Metabolism.json  bios7.f430.json
bios7.AA.json                    bios7.fa.json
bios7.Central.json               bios7.mycolate.json
bios7.LipidIV.json               bios7.peptidoglycan.json
bios7.Nucleotides.json           bios7.proteins.json
bios7.Quinones.json              bios7.sugars.json
bios7.Riboflavin.json            bios7.thf.json
bios7.Thiamine.json              proteins.json


In [251]:
cobra_model.metabolites.cpd00346_c0.summary()

PRODUCING REACTIONS -- L-Aspartate4-semialdehyde (cpd0034...)
-------------------------------------------------------------
%       FLUX  RXN ID      REACTION
----  ------  ----------  --------------------------------------------------
76%     3.72  rxn0130...  cpd00003_c0 + cpd00227_c0 --> cpd00004_c0 + cpd...
24%     1.2   rxn0164...  cpd00006_c0 + cpd00009_c0 + cpd00346_c0 <=> cpd...

CONSUMING REACTIONS -- L-Aspartate4-semialdehyde (cpd0034...)
-------------------------------------------------------------
%       FLUX  RXN ID      REACTION
----  ------  ----------  --------------------------------------------------
100%    4.91  rxn0164...  cpd00020_c0 + cpd00346_c0 --> 2 cpd00001_c0 + c...


'../bin/escher/maps/bios7.Central.html'